In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.4'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
    SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year ASC
"""

result_df = spark.sql(query)

result_df.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built ASC
"""

result_df = spark.sql(query)

result_df.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built ASC
"""

result_df = spark.sql(query)

result_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
"""

result_df = spark.sql(query)

end_time = time.time()
query_runtime = end_time - start_time

result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.047882318496704 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')


True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time_cached = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
"""

cached_result_df = spark.sql(query)

end_time_cached = time.time()
cached_query_runtime = end_time_cached - start_time_cached


cached_result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

print(f"Uncached Query Runtime: {query_runtime:.4f} seconds")
print(f"Cached Query Runtime: {cached_query_runtime:.4f} seconds")
print(f"Performance Improvement: {((query_runtime - cached_query_runtime) / query_runtime) * 100:.2f}%")

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 219.30561447143555 seconds ---
Uncached Query Runtime: 0.0789 seconds
Cached Query Runtime: 0.0668 seconds
Performance Improvement: 15.41%


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_path = "/content/home_sales_partitioned"

df.write.mode("overwrite").partitionBy("date_built").parquet(parquet_path)

print("Partitioned Parquet file has been successfully created.")

os.listdir(parquet_path)

Partitioned Parquet file has been successfully created.


['date_built=2015',
 'date_built=2017',
 '_SUCCESS',
 '._SUCCESS.crc',
 'date_built=2014',
 'date_built=2011',
 'date_built=2010',
 'date_built=2013',
 'date_built=2012',
 'date_built=2016']

In [21]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet(parquet_path)

parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [28]:
# 12. Create a temporary table for the parquet data.
parquet_df = spark.read.parquet("/content/home_sales_partitioned")

parquet_df.createOrReplaceTempView("home_sales_parquet")

print("Partitioned Parquet DataFrame is ready for querying.")

Partitioned Parquet DataFrame is ready for querying.


In [30]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time_parquet = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
"""

parquet_result_df = spark.sql(query)


end_time_parquet = time.time()
parquet_query_runtime = end_time_parquet - start_time_parquet

# Show result
parquet_result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

print(f"Uncached Query Runtime: {query_runtime:.4f} seconds")
print(f"Cached Query Runtime: {cached_query_runtime:.4f} seconds")
print(f"Parquet Query Runtime: {parquet_query_runtime:.4f} seconds")

# Calculate performance improvement
cached_improvement = ((query_runtime - cached_query_runtime) / query_runtime) * 100
parquet_improvement = ((query_runtime - parquet_query_runtime) / query_runtime) * 100

print(f"Performance Improvement from Uncached to Cached: {cached_improvement:.2f}%")
print(f"Performance Improvement from Uncached to Parquet: {parquet_improvement:.2f}%")

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 279.1990895271301 seconds ---
Uncached Query Runtime: 0.0789 seconds
Cached Query Runtime: 0.0668 seconds
Parquet Query Runtime: 0.0492 seconds
Performance Improvement from Uncached to Cached: 15.41%
Performance Improvement from Uncached to Parquet: 37.63%


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

print(f"Is home_sales still cached? {spark.catalog.isCached('home_sales')}")

Is home_sales still cached? False


In [32]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

print(f"Is home_sales still cached? {is_cached}")

Is home_sales still cached? False
